<a href="https://colab.research.google.com/github/SauryanPandey/Improved_Semantic_Search/blob/main/Improved_Semantic_Search_(Simple_Wiki_Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue May 21 08:11:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.7 MB/s eta 0:00:00


In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

if not os.path.exists(wikipedia_filepath):
    util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

  0%|          | 0.00/50.2M [00:00<?, ?B/s]

In [ ]:
with gzip.open(wikipedia_filepath, 'rt', encoding='utf-8') as f:
  documents = []
  for line in f:
    data = json.loads(line) #Converting string lines in JSON file into dictionary in Python

    documents.append(data['paragraphs'][0])

print(len(documents))

169597


In [ ]:
testing_words = ['india', 'north pole', 'nlp', 'natural language processing', 'linguistics',
                 'machine learning', 'artificial intelligence', 'cheetah', 'animal', 'jaguar']

In [ ]:
new_documents = []
for doc in documents:
  for word in testing_words:
    if(word in doc.lower()):
      new_documents.append(doc)
      break
print(len(new_documents))

5872


In [ ]:
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1') #Retrieval of top k most relevant documents
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2') #Reranking of these documents based upon similarity scores

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

#Retrieval with Bi-Encoders

In [ ]:
new_documents[:5]

["The integumentary system is everything covering the outside of an animal's body. This account is written mostly with people in mind, but it applies more widely.",
 'The electric eel, "Electrophorus electricus", is a species of fish. It can weigh up to six pounds. It can give electric shocks of up to six hundred fifty watts of electricity. The animal uses these shocks both for hunting and to defend itself. It lives in the stagnant muddy river bottoms of the Orinoco and the Amazon, and uses low-voltage electric fields to find its prey.',
 'The term Far East means countries of East Asia. It became used in the English language during the period of the British Empire for lands to the east of British India.',
 "A chakra is an energy center in the human body as taught in Indian yoga and some related cultures. Chakras are also believed in by some New Age adherents. Chakra means ‘Aneel or wheel' in Sanskrit.",
 'The Simputer is a small, cheap device, similar to personal digital assistant (pal

In [ ]:
documents_embeddings = bi_encoder.encode(new_documents)

Batches:   0%|          | 0/184 [00:00<?, ?it/s]

In [ ]:
documents_embeddings[0].reshape(1,-1)

array([[ 2.43610446e-03,  3.29437926e-02,  4.70606014e-02,
        -7.68167479e-03,  1.48560539e-01, -7.82975107e-02,
         2.99415477e-02,  1.36641292e-02,  1.53808743e-02,
         1.40475079e-01,  2.35609505e-02, -8.67463648e-02,
         2.46762913e-02,  8.08474980e-03, -1.67811643e-02,
        -6.50802329e-02, -3.48654017e-02, -7.70691840e-04,
        -5.02524003e-02,  7.39627331e-03, -2.48694886e-02,
         6.15889803e-02, -2.88300738e-02,  8.74973182e-03,
        -1.11718066e-01, -3.45828454e-03, -4.96662334e-02,
        -5.51604889e-02, -3.83206941e-02, -6.52881041e-02,
        -4.57847957e-03, -4.55709547e-02,  5.72044179e-02,
         3.42551805e-02, -4.65118140e-03, -2.70470697e-02,
         3.08932122e-02, -2.60512847e-02, -5.11268862e-02,
         3.61177027e-02,  7.31430808e-03,  2.07178947e-02,
         2.00569984e-02,  2.59086005e-02,  8.13343152e-02,
         4.40371670e-02, -1.29732475e-01, -6.40622452e-02,
         8.23869649e-03, -3.58666468e-04,  7.39453956e-0

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def relevant_docs(query, count):
  query = query.lower()
  query_embedding = bi_encoder.encode(query)
  a = []
  for d in documents_embeddings:
    sim = cosine_similarity(d.reshape(1,-1), query_embedding.reshape(1,-1))
    a.append(sim[0][0])
  a = np.array(a)
  cnt = 0
  idx = []
  while cnt<count:
    max_idx = np.argmax(a)
    idx.append(max_idx)
    a = np.delete(a, max_idx)
    cnt+=1

  return idx

In [ ]:
query1 = 'What is the financial capital of India?'
idx1 = relevant_docs(query1, 100)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
new_documents[idx1[0]] #Most Relevant Document

'The National Stock Exchange of India Limited (NSE), is a Mumbai-based stock exchange. It is the biggest stock exchange in India and the third biggest in the world in terms of amounts of transactions. NSE is mutually-owned by a set of leading financial institutions, banks, insurance companies and other financial intermediaries in India but its ownership and management operate as separate groups. As of 2006, the NSE VSAT terminals, 2799 in total, cover more than 1500 cities across India. In July 2007, the NSE had a total market capitalization of 42,74,509 crore INR making it the second-largest stock market in South Asia in terms of market-capitalization.'

#Re-ranking with Cross-Encoders


In [ ]:
def sentence_pairs(query, documents, idx):
    qd = []
    topk_similar_docs = []
    for i in idx:
        qd.append([query, documents[i]])
        topk_similar_docs.append(documents[i])
    return qd, topk_similar_docs

In [ ]:
query_doc_pairs, topk_similar_docs = sentence_pairs(query1, new_documents, idx1)

In [ ]:
similarity_scores = cross_encoder.predict(query_doc_pairs, show_progress_bar = True, convert_to_numpy = True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
print(similarity_scores)

[  0.24508783  -8.946733     3.5322978  -10.752103   -11.071547
  -9.461139   -11.213962    -1.2995698  -11.128358   -11.04509
  -8.20163    -10.412138   -10.692642   -11.175376    -5.297061
  -3.2702558   -6.889288    -9.257908   -11.154494   -10.16055
 -10.817291    -6.994435   -11.050814   -10.215964     3.5322978
  -4.1622624  -11.078227   -10.983159   -11.0834675  -11.092703
 -10.960367   -11.136716    -9.939579   -10.597785   -10.64696
 -10.606368   -11.06457    -10.930551   -10.851954   -10.768676
 -11.080635   -10.936771    -9.623537   -10.680507   -10.987901
 -10.351691   -11.091391   -11.055042    -6.7052774  -11.089347
 -10.324846   -10.07918    -11.111828   -11.07732    -11.1257305
 -10.31979    -10.948334   -10.962687   -10.969747   -11.041257
 -10.908257   -10.651577    -9.114443   -10.413861   -10.9909525
  -6.50263    -10.940133   -10.970784   -10.879505   -11.095625
 -11.087531   -11.019806   -10.229019   -10.977057   -11.036898
  -9.13717    -10.525365    -8.571441   

In [ ]:
best_idx = np.argmax(similarity_scores)
best_idx

2

In [ ]:
topk_similar_docs[best_idx] #Most Relevant Doc after Re-ranking

"Mumbai (previously known as Bombay until 1996) is a natural harbor on the west coast of India, and is the capital city of Maharashtra state. It is India's largest city, and one of the world's most populous cities. It is the financial capital of India. The city is the second most-populous in the world. It has approximately 13 million people. Along with the neighboring cities of Navi Mumbai and Thane, it forms the world's 4th largest urban agglomeration. They have around 19.1 million people."